In [1]:
import numpy as np
import pandas as pd
import os
import re

import nltk.corpus
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from numpy.linalg import norm

import time

#from multiprocessing.pool import ThreadPool as Pool
#from multiprocessing import *


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
os.chdir(r"C:\Users\swani\Python Projects\Case Bucket\Data")

In [3]:
data = pd.read_excel('N-Grams Data All Articles.xlsx')
length = int(len(data))
print('Data Length: %d' %(length))
print(data.head())

Data Length: 125831
   Unnamed: 0      Name  Article Number                          N-Grams
0           0  20467935          538672            vhdl get csgmss error
1           1  20467935          538672            get csgmss error data
2           2  20467935          538672           csgmss error data type
3           3  20467935          538672       error data type unresolved
4           4  20467935          538672  data type unresolved std_ulogic


In [4]:
searchTerms = pd.read_excel('XceliumCD_Data.xlsx')
length = int(len(searchTerms))
print('Data Length: %d' %(length))
print(searchTerms.head())

df = pd.read_excel('ALl-keywords-2021.xlsx')
exceptions = df["Keywords Flagged by IT"].tolist()

Data Length: 2059
  Product                                         Search Term  \
0  Xcelium                           probe depth not working    
1  Xcelium  Using environment variables in a timing check ...   
2  Xcelium                              Simulation profiling    
3  Xcelium                               timescale overriding   
4  Xcelium                             conditionally stopping   

              N-Grams Search Results                Unnamed: 3  \
0             probe create top depth      depth 3 probe create   
1             used timing file tfile  option timing file tfile   
2          dump profiling data xmsim   dpi data type profiling   
3  timescale value module overriding                       NaN   
4       conditionally split log file                       NaN   

                        Unnamed: 4  KB Articles  Unnamed: 6  Unnamed: 7  \
0                              NaN   20489076.0  20493943.0         NaN   
1                              NaN   2048567

In [32]:
ngrmasList = []
kbArticles = []
cosineScore = []

extreme_start = time.time()

k = 0
for term in searchTerms['Search Term']:
    print(str(k)+']')
    start = time.time()
        
    a = term.lower()
    a = a.replace('[^\w\s]','')
    stop_words = stopwords.words('english')
    a = [word for word in a.split() if word not in (stop_words)]
    a = ' '.join(a)
    a = [word for word in a.split() if word not in (exceptions)]
    a = ' '.join(a)
    lmtzr = WordNetLemmatizer()
    lemmatized = [lmtzr.lemmatize(word) for word in word_tokenize(a)]
    a = ' '.join(lemmatized)
    print('Processed input: %s' %(a))

    ngrams = data['N-Grams']
    ngrams_list = ngrams.tolist()
    ngrams_list.append(a)

    vectorizer = TfidfVectorizer().fit_transform(ngrams_list)
    input_vector = vectorizer[-1]
    vectorizer = vectorizer[:-1]

    from sklearn.metrics.pairwise import linear_kernel
    cosine_similarities = linear_kernel(vectorizer[:], input_vector).flatten()

    sort_cs = np.sort(cosine_similarities)[::-1]
    top3_values = sort_cs[:3]
        
    if (any(i >= 0.8 for i in top3_values) is True):
        cosineScore.append(top3_values)

    else:
        top3_values = [0,0,0]
        cosineScore.append(top3_values)

    print(top3_values)
    
    indices_list = []
    
    if (all(i == 0 for i in top3_values) is True):
        indices_list = [0,0,0]
    else:
        for value in top3_values:
            max_list = [i for i,val in enumerate(cosine_similarities) if val==value]
            indices_list.append(max_list[0])
        
    print('The indices of the n-grams: %a'%indices_list)

    if (indices_list == [0,0,0]):
            kbArticles.append([])
            print('The KB Articles: None')
    else:
        lst = [data.iloc[i]['Name'] for i in indices_list]
        KB_Articles = np.unique(lst)
        print('The KB Articles: %a'%KB_Articles)
        kbArticles.append(KB_Articles)
        unique_indices = [lst.index(i) for i in KB_Articles]
    
    op_ngrams = []
    if (indices_list == [0,0,0]):
        op_ngrams.append('')
    else:
        for i in unique_indices:
            index = indices_list[i]
            op_ngrams.append(ngrams_list[index])
    print(op_ngrams)
    ngrmasList.append(op_ngrams)
    
    end = time.time()
    print('Time Taken to Respond: %d seconds' %(end-start))
    print('')
    k = k+1
    
extreme_end = time.time()
print('Time Taken for the whole code to Respond: %d seconds' %(extreme_end-extreme_start))

0]
Processed input: probe depth working
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

1]
Processed input: using environment variable timing check tfile
[0.85193251 0.59995454 0.59915412]
The indices of the n-grams: [123534, 34987, 27805]
The KB Articles: array([20452070, 20485672, 20486602], dtype=int64)
['timing check using tfile', 'used timing file tfile', 'option timing file tfile']
Time Taken to Respond: 1 seconds

2]
Processed input: profiling
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

3]
Processed input: timescale overriding
[0.86433557 0.8599573  0.77640541]
The indices of the n-grams: [81427, 81443, 81442]
The KB Articles: array([20405661], dtype=int64)
['timescale value module overriding']
Time Taken to Respond: 1 seconds

4]
Processed input: conditionally stopping
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken 

[1. 1. 1.]
The indices of the n-grams: [20565, 20565, 20565]
The KB Articles: array([20480344], dtype=int64)
['sv_seghandler trapno 1 addr']
Time Taken to Respond: 1 seconds

46]
Processed input: dlcsmd worklib.specman : ncrunsv
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

47]
Processed input: getting undefined symbol abc.so file even though definition present file
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

48]
Processed input: verdi
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

49]
Processed input: running
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

50]
Processed input: trying verilog & vhdl file compile virtuoso 6.17 getting following
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond:

[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

92]
Processed input: failed retrieve input data area evcd file
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

93]
Processed input: updating 19.03.002 imc tool give version mismatch
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

94]
Processed input: error : error processing stack frame ( 37 ) - skipping rest frame !
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

95]
Processed input: crash : xmsim : * f , scsig : signal sigfpe raised systemc library applicat
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

96]
Processed input: vmanager ( imc ) tool appears missing .
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']

[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

137]
Processed input: constant generated generate statement re-generated power-up lp corrup
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

138]
Processed input: http : //support.cadence.com/apex/articleattachmentportal ? id=a1od0000000nw0aeae & pagename=articleconte
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

139]
Processed input: extract file list ( with absolute path ) verilog/systemverilog source contributing
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

140]
Processed input: incremental elaba
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

141]
Processed input: multi clock property concurrent assertion
[0, 0, 0]
The indices of t

[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

182]
Processed input: merge coverage result ?
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

183]
Processed input: error mean ? vm_scan.pl : log scan match file.\nlocal_log.log\nvm_s
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

184]
Processed input: system verilog top am
[0.82826207 0.59703012 0.58223581]
The indices of the n-grams: [47165, 47164, 82971]
The KB Articles: array([20416947, 20489621], dtype=int64)
['system verilog sv system', 'am design system verilog']
Time Taken to Respond: 1 seconds

185]
Processed input: if ( a === 1'bx ) working gls
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 1 seconds

186]
Processed input: sdf retain question
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB A

...

[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1825]
Processed input: vcs loopreport
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1826]
Processed input: view delta cycle waveform
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1827]
Processed input: cuvmur error
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1828]
Processed input: specman , e , override , test , constraint , sequence
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1829]
Processed input: training
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1830]
Processed input: training
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Ta

[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1874]
Processed input: delta event waveform
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1875]
Processed input: expression coverage data imc
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1876]
Processed input: write one tcl procedure wait force ? f
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1877]
Processed input: write one tcl procedure wait force ?
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1878]
Processed input: trtovf : simulated time overflow .
[1.         1.         0.85590119]
The indices of the n-grams: [89202, 89202, 89201]
The KB Articles: array([1840958], dtype=int64)
['trtovf simulated time overflow']
Time Taken t

[0.84783747 0.80923908 0.78979543]
The indices of the n-grams: [37698, 11662, 11659]
The KB Articles: array([20484416, 20487547], dtype=int64)
['tchecks 4 annotated 10000', 'disabled tchecks 0 annotated']
Time Taken to Respond: 0 seconds

1919]
Processed input: q < = rising_edge ( clk ) ;
[0.90153641 0.89172029 0.83143356]
The indices of the n-grams: [29704, 119926, 119928]
The KB Articles: array([20444574, 20483101], dtype=int64)
['clk begin rising_edge clk', 'rising_edge clk q end']
Time Taken to Respond: 0 seconds

1920]
Processed input: power bar simvision
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1921]
Processed input: crash pdk srams
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1922]
Processed input: * f , svbdupi
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1923]
Processed input: loopã�®

[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1964]
Processed input: xmsim csi
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1965]
Processed input: def param msie
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1966]
Processed input: get last n bit long double systemc
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1967]
Processed input: float modulus systemc
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

1968]
Processed input: interr
[0.93767946 0.80338282 0.79081442]
The indices of the n-grams: [69605, 69604, 11939]
The KB Articles: array([20484453, 20495272], dtype=int64)
['f interr error xrun', 'interr xmsim f interr']
Time Taken to Respond: 0 seconds

1969]
Processed input: ov

[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

2013]
Processed input: get source file signal path
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

2014]
Processed input: disable label
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

2015]
Processed input: move xcelium.d particular directory
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

2016]
Processed input: dfile
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

2017]
Processed input: axi vip documentation usage
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Time Taken to Respond: 0 seconds

2018]
Processed input: advanced profiling
[0, 0, 0]
The indices of the n-grams: [0, 0, 0]
The KB Articles: None
['']
Ti

In [35]:
datatoexcel = pd.ExcelWriter('test.xlsx')
  
df = pd.DataFrame(kbArticles)
# write DataFrame to excel
df.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()

In [36]:
df.shape

(2059, 3)

In [33]:
df = pd.DataFrame(kbArticles)

In [34]:
df

,0,1,2
0,NaN,NaN,NaN
1,20452070.0,20485672.0,20486602.0
2,NaN,NaN,NaN
3,20405661.0,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
2054,NaN,NaN,NaN
2055,NaN,NaN,NaN
2056,NaN,NaN,NaN
2057,NaN,NaN,NaN
